In [47]:
import sys
import os
import lzma

In [48]:
import catboost
import pandas


In [49]:
cancer_data_dir = '/home/leron/projects/cancer/data'
dump_dir = os.path.join(cancer_data_dir, 'bcDump/example15bmc')
merged_path = os.path.join(dump_dir, 'ex15bmcMerged.csv.xz')
bmc_all_path = os.path.join(dump_dir, 'bmc15mldata1.csv')

In [124]:
dtype = {'DFS': pandas.Int64Dtype(),
         'pCR': pandas.Int64Dtype(),
         'RFS': pandas.Int64Dtype(), 
         'DFS': pandas.Int64Dtype(), 
         'posOutcome': pandas.Int64Dtype()}

In [125]:
bmc = pandas.read_csv(bmc_all_path, dtype=dtype)
bmc = bmc.sort_values(by='patient_ID')

In [126]:
set(bmc.study.to_list())

{'study_12093_GPL96_all-bmc15',
 'study_1379_GPL1223_all-bmc15',
 'study_16391_GPL570_all-bmc15',
 'study_16446_GPL570_all-bmc15',
 'study_17705_GPL96_JBI_Tissue_BC_Tamoxifen-bmc15',
 'study_17705_GPL96_MDACC_Tissue_BC_Tamoxifen-bmc15',
 'study_19615_GPL570_all-bmc15',
 'study_20181_GPL96_all-bmc15',
 'study_20194_GPL96_all-bmc15',
 'study_2034_GPL96_all-bmc15',
 'study_22226_GPL1708_all-bmc15',
 'study_22358_GPL5325_all-bmc15',
 'study_25055_GPL96_MDACC_M-bmc15',
 'study_25065_GPL96_MDACC-bmc15',
 'study_25065_GPL96_USO-bmc15',
 'study_32646_GPL570_all-bmc15',
 'study_9893_GPL5049_all-bmc15'}

In [127]:
merged = pandas.read_csv(lzma.open(merged_path))

In [128]:
merged.head(5)

,patient_ID,MAGEA12,MAGEA11,KLF1,ADH7,MSH4,BIRC3,AKR1C4,GBX2,GCGR,...,ZNF80,ZNF83,ZNF84,ZNF91,ZNHIT2,ZSCAN2,ZXDC,ZYX,ZZEF1,ZZZ3
0,22449,-0.118953,1.180345,0.252643,-0.262987,0.142903,0.167314,0.498846,0.774632,0.104353,...,-1.564143,0.466733,0.827552,-0.617981,0.303161,1.260602,-0.217995,0.219529,0.389849,1.313703
1,22450,0.423693,-0.922374,-1.202192,-0.105451,-0.061571,-0.093231,-0.095550,-0.481403,-0.214238,...,0.711752,0.358388,0.037911,2.304784,0.328942,-1.028791,-0.850002,-0.292574,-0.068982,0.722123
2,22451,-0.239183,-0.733389,0.523791,-0.081958,-0.004635,-0.008094,0.268636,-0.614192,0.027471,...,-0.011786,-0.474762,-0.349981,-0.097197,0.100946,-0.554700,-0.367363,0.094464,-0.372665,-0.790771
3,22452,0.500445,-0.177686,-0.216638,-0.130850,-0.261039,-0.048521,1.479664,-0.100120,0.233178,...,0.757255,0.590212,0.060150,2.287583,-0.108866,-1.132500,-0.106976,-0.216267,0.393671,-0.027349
4,22453,-0.609235,0.259494,-0.071802,0.027963,0.162509,0.112654,-0.239435,0.229737,-0.132271,...,0.407159,0.570637,0.851658,-0.412950,0.105692,-1.047445,0.084480,-0.224081,-0.021074,0.764555


In [129]:
train_feat = merged[merged.patient_ID.isin(bmc.patient_ID)]

In [130]:
train_feat = train_feat.sort_values(by='patient_ID')

In [131]:
feature_columns = train_feat.columns.to_list()[1:] + ['radio', 'surgery', 'chemo', 'hormone']
label_columns = ['pCR', 'RFS', 'DFS', 'posOutcome']

In [132]:
assert(bmc.patient_ID.to_list() == train_feat.patient_ID.to_list())

In [133]:
merged_train = pandas.merge(train_feat, bmc, left_on='patient_ID', right_on='patient_ID')

In [134]:
bmc.pCR.min()

0

In [135]:
merged_train

,patient_ID,MAGEA12,MAGEA11,KLF1,ADH7,MSH4,BIRC3,AKR1C4,GBX2,GCGR,...,ZZZ3,study,radio,surgery,chemo,hormone,pCR,RFS,DFS,posOutcome
0,22449,-0.118953,1.180345,0.252643,-0.262987,0.142903,0.167314,0.498846,0.774632,0.104353,...,1.313703,study_1379_GPL1223_all-bmc15,0,NaN,0,1,<NA>,<NA>,0,0
1,22450,0.423693,-0.922374,-1.202192,-0.105451,-0.061571,-0.093231,-0.095550,-0.481403,-0.214238,...,0.722123,study_1379_GPL1223_all-bmc15,0,NaN,0,1,<NA>,<NA>,0,0
2,22451,-0.239183,-0.733389,0.523791,-0.081958,-0.004635,-0.008094,0.268636,-0.614192,0.027471,...,-0.790771,study_1379_GPL1223_all-bmc15,0,NaN,0,1,<NA>,<NA>,0,0
3,22452,0.500445,-0.177686,-0.216638,-0.130850,-0.261039,-0.048521,1.479664,-0.100120,0.233178,...,-0.027349,study_1379_GPL1223_all-bmc15,0,NaN,0,1,<NA>,<NA>,0,0
4,22453,-0.609235,0.259494,-0.071802,0.027963,0.162509,0.112654,-0.239435,0.229737,-0.132271,...,0.764555,study_1379_GPL1223_all-bmc15,0,NaN,0,1,<NA>,<NA>,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,809294,0.078549,0.012204,1.447419,0.269271,0.756916,0.043398,0.593809,0.498024,-0.375959,...,0.930898,study_32646_GPL570_all-bmc15,0,NaN,1,0,1,<NA>,<NA>,1
2221,809295,0.462802,-0.124217,0.010072,-0.131620,0.198549,-0.274135,0.683212,-0.267721,-0.082954,...,0.718979,study_32646_GPL570_all-bmc15,0,NaN,1,0,1,<NA>,<NA>,1
2222,809296,1.237760,-0.087561,-0.061712,0.409251,0.268137,-0.203154,0.077432,0.092998,0.319132,...,-0.072171,study_32646_GPL570_all-bmc15,0,NaN,1,0,0,<NA>,<NA>,0
2223,809297,0.375490,-0.742451,-0.235492,0.205205,0.209221,-0.529743,0.022536,-0.695793,-0.207093,...,-0.447401,study_32646_GPL570_all-bmc15,0,NaN,1,0,0,<NA>,<NA>,0
